# Analyse Complète: Market Data → Heston Calibration → Monte Carlo → IV Surfaces

Ce notebook effectue une analyse complète en 5 étapes:
1. **Téléchargement des données** de yfinance
2. **Heatmap des prix market** (grille S × K)
3. **IV Surface des données market** en 3D
4. **Calibration Heston** via réseau de neurones PyTorch
5. **Heatmap des prix Heston** (Monte Carlo)
6. **IV Surface BS** inversée à partir des prix Heston en 3D

## 1. Imports et Configuration

In [1]:
from IPython.display import display
from __future__ import annotations

import math
from typing import Dict, Tuple

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import torch
import yfinance as yf

from heston_torch import HestonParams, carr_madan_call_torch

torch.set_default_dtype(torch.float64)
DEVICE = torch.device("cpu")
MIN_IV_MATURITY = 0.1

print("✓ Imports réussis")

✓ Imports réussis


## 2. Configuration des Paramètres

In [2]:
# Paramètres de marché
TICKER = "SPY"
RF_RATE = 0.02
MATURITY = 1.0  # 1 an

# Paramètres de la grille (S0 ± 10, step de 1)
SPAN = 10.0
STEP = 1.0

# Paramètres calibration (augmenté pour utiliser plus de données)
MAX_POINTS = 1000  # Augmenté pour profiter de toutes les données
MAX_ITERS = 150    # Plus d'itérations pour meilleure convergence
LR = 5e-3

# Paramètres Monte Carlo
N_PATHS = 50000
N_STEPS = 100

print(f"Configuration:")
print(f"  Ticker: {TICKER}")
print(f"  Maturity: {MATURITY} ans")
print(f"  Grille: S0 ± {SPAN}, step = {STEP}")
print(f"  Monte Carlo: {N_PATHS:,} trajectoires, {N_STEPS} pas")

Configuration:
  Ticker: SPY
  Maturity: 1.0 ans
  Grille: S0 ± 10.0, step = 1.0
  Monte Carlo: 50,000 trajectoires, 100 pas


## 3. Fonctions Utilitaires

In [3]:
def fetch_spot(symbol: str) -> float:
    """Récupère le prix spot actuel."""
    ticker = yf.Ticker(symbol)
    hist = ticker.history(period="1d")
    if hist.empty:
        raise RuntimeError("Unable to retrieve spot price.")
    return float(hist["Close"].iloc[-1])


def download_all_options(symbol: str, years_ahead: float = 2.5) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Télécharge TOUTES les options disponibles pour maximiser les données d'entraînement."""
    ticker = yf.Ticker(symbol)
    spot = fetch_spot(symbol)
    expirations = ticker.options
    
    if not expirations:
        raise RuntimeError(f"No option expirations found for {symbol}")
    
    now = pd.Timestamp.utcnow().tz_localize(None)
    limit_date = now + pd.Timedelta(days=365 * years_ahead)
    
    calls_data = []
    puts_data = []
    
    print(f"Téléchargement de toutes les expirations jusqu'à {years_ahead} ans...")
    for idx, exp in enumerate(expirations):
        exp_dt = pd.Timestamp(exp)
        if exp_dt > limit_date:
            continue
            
        T = max((exp_dt - now).total_seconds() / (365.0 * 24 * 3600), 0.0)
        if T < 0.01:  # Ignorer les expirations très proches
            continue
        
        try:
            chain = ticker.option_chain(exp)
            
            # Calls
            for _, row in chain.calls.iterrows():
                K = float(row["strike"])
                # Filtre: seulement les strikes dans S0 ± 50 pour réduire le bruit
                if abs(K - spot) <= 50:
                    calls_data.append({
                        "S0": spot,
                        "K": K,
                        "T": T,
                        "C_mkt": float(row["lastPrice"]),
                        "iv_market": float(row.get("impliedVolatility", float("nan"))),
                        "option_type": "call",
                        "expiry": exp
                    })
            
            # Puts
            for _, row in chain.puts.iterrows():
                K = float(row["strike"])
                if abs(K - spot) <= 50:
                    puts_data.append({
                        "S0": spot,
                        "K": K,
                        "T": T,
                        "P_mkt": float(row["lastPrice"]),
                        "iv_market": float(row.get("impliedVolatility", float("nan"))),
                        "option_type": "put",
                        "expiry": exp
                    })
            
            if (idx + 1) % 5 == 0:
                print(f"  Processed {idx + 1}/{len(expirations)} expirations...")
                
        except Exception as e:
            print(f"  Warning: Failed to download {exp}: {e}")
            continue
    
    calls_df = pd.DataFrame(calls_data)
    puts_df = pd.DataFrame(puts_data)
    
    # Nettoyer les données
    calls_df = calls_df[calls_df["C_mkt"] > 0]
    puts_df = puts_df[puts_df["P_mkt"] > 0]
    calls_df = calls_df[~calls_df["iv_market"].isna()]
    puts_df = puts_df[~puts_df["iv_market"].isna()]
    
    return calls_df, puts_df


def plot_heatmap_2d(data: np.ndarray, K_grid: np.ndarray, S_grid: np.ndarray, title: str, zlabel: str = "Value"):
    """Crée une heatmap 2D."""
    fig = go.Figure(
        data=go.Heatmap(
            z=data,
            x=K_grid,
            y=S_grid,
            colorscale="Viridis",
            colorbar=dict(title=zlabel),
        )
    )
    fig.update_layout(
        title=title,
        xaxis_title="Strike K",
        yaxis_title="Spot S",
        width=700,
        height=600,
    )
    return fig


def plot_surface_3d(S_grid: np.ndarray, K_grid: np.ndarray, data: np.ndarray, title: str, zlabel: str = "Value"):
    """Crée une surface 3D."""
    K_mesh, S_mesh = np.meshgrid(K_grid, S_grid)
    
    fig = go.Figure(
        data=go.Surface(
            x=K_mesh,
            y=S_mesh,
            z=data,
            colorscale="Viridis",
            colorbar=dict(title=zlabel),
        )
    )
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title="Strike K",
            yaxis_title="Spot S",
            zaxis_title=zlabel,
        ),
        width=800,
        height=700,
    )
    return fig

print("✓ Fonctions utilitaires définies")

✓ Fonctions utilitaires définies


## 4. Fonctions Black-Scholes

In [4]:
def bs_price(S0: float, K: float, T: float, vol: float, r: float, option_type: str = "call") -> float:
    """Prix Black-Scholes."""
    if T <= 0.0 or vol <= 0.0 or S0 <= 0.0 or K <= 0.0:
        if option_type == "call":
            return max(0.0, S0 - K * math.exp(-r * T))
        else:
            return max(0.0, K * math.exp(-r * T) - S0)
    
    sqrt_T = math.sqrt(T)
    d1 = (math.log(S0 / K) + (r + 0.5 * vol * vol) * T) / (vol * sqrt_T)
    d2 = d1 - vol * sqrt_T
    
    nd1 = 0.5 * (1.0 + math.erf(d1 / math.sqrt(2.0)))
    nd2 = 0.5 * (1.0 + math.erf(d2 / math.sqrt(2.0)))
    
    if option_type == "call":
        return S0 * nd1 - K * math.exp(-r * T) * nd2
    else:
        return K * math.exp(-r * T) * (1 - nd2) - S0 * (1 - nd1)


def implied_vol_from_price(
    price: float, S0: float, K: float, T: float, r: float, option_type: str = "call",
    tol: float = 1e-6, max_iter: int = 100
) -> float:
    """Calcule la volatilité implicite par bissection."""
    if option_type == "call":
        intrinsic = max(0.0, S0 - K * math.exp(-r * T))
    else:
        intrinsic = max(0.0, K * math.exp(-r * T) - S0)
    
    if price <= intrinsic + 1e-12:
        return 0.0
    
    low, high = 1e-6, 3.0
    p_high = bs_price(S0, K, T, high, r, option_type)
    
    while p_high < price and high < 10.0:
        high *= 2.0
        p_high = bs_price(S0, K, T, high, r, option_type)
    
    if p_high < price:
        return float("nan")
    
    for _ in range(max_iter):
        mid = 0.5 * (low + high)
        p_mid = bs_price(S0, K, T, mid, r, option_type)
        
        if abs(p_mid - price) < tol:
            return mid
        
        if p_mid > price:
            high = mid
        else:
            low = mid
    
    return 0.5 * (low + high)

print("✓ Fonctions Black-Scholes définies")

✓ Fonctions Black-Scholes définies


## 5. Fonctions de Calibration Heston

In [5]:
def prices_from_unconstrained(
    u: torch.Tensor, S0_t: torch.Tensor, K_t: torch.Tensor, T_t: torch.Tensor, r: float, q: float
) -> torch.Tensor:
    params = HestonParams.from_unconstrained(u[0], u[1], u[2], u[3], u[4])
    prices = []
    for S0_i, K_i, T_i in zip(S0_t, K_t, T_t):
        price_i = carr_madan_call_torch(S0_i, r, q, T_i, params, K_i)
        prices.append(price_i)
    return torch.stack(prices)


def loss(u: torch.Tensor, S0_t: torch.Tensor, K_t: torch.Tensor, T_t: torch.Tensor, C_mkt_t: torch.Tensor, r: float, q: float) -> torch.Tensor:
    model_prices = prices_from_unconstrained(u, S0_t, K_t, T_t, r, q)
    return torch.mean((model_prices - C_mkt_t) ** 2)


def calibrate_heston(calls_df: pd.DataFrame, r: float, q: float, max_points: int, max_iters: int, lr: float) -> dict:
    """Calibre les paramètres Heston sur les calls."""
    df = calls_df.copy()
    df = df[df["T"] >= MIN_IV_MATURITY]
    df = df[df["C_mkt"] > 0]
    df = df.sample(n=min(len(df), max_points), random_state=42)

    S0_t = torch.tensor(df["S0"].values, dtype=torch.float64, device=DEVICE)
    K_t = torch.tensor(df["K"].values, dtype=torch.float64, device=DEVICE)
    T_t = torch.tensor(df["T"].values, dtype=torch.float64, device=DEVICE)
    C_mkt_t = torch.tensor(df["C_mkt"].values, dtype=torch.float64, device=DEVICE)

    u = torch.tensor([0.0, 0.0, 0.0, 0.0, 0.0], dtype=torch.float64, device=DEVICE, requires_grad=True)
    optimizer = torch.optim.Adam([u], lr=lr)

    print(f"Calibration sur {len(df)} points...")
    for it in range(max_iters):
        optimizer.zero_grad()
        L = loss(u, S0_t, K_t, T_t, C_mkt_t, r, q)
        L.backward()
        optimizer.step()
        
        if (it + 1) % 20 == 0:
            print(f"  Iter {it+1}/{max_iters} | Loss = {L.item():.6e}")

    params_final = HestonParams.from_unconstrained(u[0], u[1], u[2], u[3], u[4])
    calib = {
        "kappa": float(params_final.kappa.cpu()),
        "theta": float(params_final.theta.cpu()),
        "sigma": float(params_final.sigma.cpu()),
        "rho": float(params_final.rho.cpu()),
        "v0": float(params_final.v0.cpu()),
    }
    
    return calib


def params_from_calib(calib: dict) -> HestonParams:
    """Convertit dict en HestonParams."""
    return HestonParams(
        kappa=torch.tensor(calib["kappa"], dtype=torch.float64, device=DEVICE),
        theta=torch.tensor(calib["theta"], dtype=torch.float64, device=DEVICE),
        sigma=torch.tensor(calib["sigma"], dtype=torch.float64, device=DEVICE),
        rho=torch.tensor(calib["rho"], dtype=torch.float64, device=DEVICE),
        v0=torch.tensor(calib["v0"], dtype=torch.float64, device=DEVICE),
    )

print("✓ Fonctions de calibration définies")

✓ Fonctions de calibration définies


## 6. Fonctions Monte Carlo Heston

In [6]:
def heston_monte_carlo_price(
    S0: float, K: float, T: float, r: float, q: float,
    params: HestonParams, n_paths: int, n_steps: int, option_type: str = "call"
) -> float:
    """Price option avec Monte Carlo Heston."""
    dt = T / n_steps
    sqrt_dt = np.sqrt(dt)
    
    kappa = float(params.kappa.cpu())
    theta = float(params.theta.cpu())
    sigma = float(params.sigma.cpu())
    rho = float(params.rho.cpu())
    v0 = float(params.v0.cpu())
    
    S = np.ones(n_paths) * S0
    v = np.ones(n_paths) * v0
    
    for _ in range(n_steps):
        Z1 = np.random.standard_normal(n_paths)
        Z2 = rho * Z1 + np.sqrt(1 - rho**2) * np.random.standard_normal(n_paths)
        
        v_sqrt = np.sqrt(np.maximum(v, 0))
        v = v + kappa * (theta - v) * dt + sigma * v_sqrt * sqrt_dt * Z2
        v = np.maximum(v, 0)
        
        S = S * np.exp((r - q - 0.5 * v) * dt + v_sqrt * sqrt_dt * Z1)
    
    if option_type == "call":
        payoff = np.maximum(S - K, 0)
    else:
        payoff = np.maximum(K - S, 0)
    
    return np.exp(-r * T) * np.mean(payoff)

print("✓ Fonctions Monte Carlo définies")

✓ Fonctions Monte Carlo définies


## 7. ÉTAPE 1: Téléchargement des Données Market

In [7]:
print("=" * 80)
print("ÉTAPE 1: Téléchargement des données market")
print("=" * 80)

# Télécharger TOUTES les options disponibles
calls_df, puts_df = download_all_options(TICKER, years_ahead=2.5)
S0 = fetch_spot(TICKER)

print(f"\n✓ Spot actuel: ${S0:.2f}")
print(f"✓ {len(calls_df)} calls téléchargés (toutes maturités)")
print(f"✓ {len(puts_df)} puts téléchargés (toutes maturités)")

# Statistiques des données
print(f"\nStatistiques des données téléchargées:")
print(f"  Calls - Maturités: {calls_df['T'].min():.2f} à {calls_df['T'].max():.2f} ans")
print(f"  Calls - Strikes: ${calls_df['K'].min():.2f} à ${calls_df['K'].max():.2f}")
print(f"  Puts  - Maturités: {puts_df['T'].min():.2f} à {puts_df['T'].max():.2f} ans")
print(f"  Puts  - Strikes: ${puts_df['K'].min():.2f} à ${puts_df['K'].max():.2f}")

# Créer la grille S × K pour les heatmaps (S0 ± 10, step 1)
S_min, S_max = S0 - SPAN, S0 + SPAN
K_min, K_max = S0 - SPAN, S0 + SPAN
S_grid = np.arange(S_min, S_max + STEP, STEP)
K_grid = np.arange(K_min, K_max + STEP, STEP)

print(f"\n✓ Grille pour heatmaps: {len(S_grid)} × {len(K_grid)} = {len(S_grid) * len(K_grid)} points")
print(f"  S: {S_min:.1f} à {S_max:.1f} (step {STEP})")
print(f"  K: {K_min:.1f} à {K_max:.1f} (step {STEP})")

ÉTAPE 1: Téléchargement des données market

✓ Spot actuel: $660.45
✓ Maturité trouvée: 1.08 ans
✓ 145 calls téléchargés
✓ 130 puts téléchargés

✓ Grille créée: 21 × 21 = 441 points
  S: 650.5 à 670.5
  K: 650.5 à 670.5


## 8. ÉTAPE 2: Heatmap des Prix Market

In [8]:
print("=" * 80)
print("ÉTAPE 2: Construction des heatmaps de prix market")
print("=" * 80)

# Pour les heatmaps, on va agréger les prix market sur la grille K
# On prend la maturité la plus proche de 1 an pour l'affichage
target_T = 1.0
calls_near_1y = calls_df[np.abs(calls_df['T'] - target_T) < 0.2].copy()
puts_near_1y = puts_df[np.abs(puts_df['T'] - target_T) < 0.2].copy()

# Grouper par strike et prendre la moyenne des prix
call_prices_by_K = calls_near_1y.groupby('K')['C_mkt'].mean().to_dict()
put_prices_by_K = puts_near_1y.groupby('K')['P_mkt'].mean().to_dict()

# Créer les heatmaps (prix à S0 seulement, autres lignes vides)
call_prices_market = np.full((len(S_grid), len(K_grid)), np.nan)
put_prices_market = np.full((len(S_grid), len(K_grid)), np.nan)

s0_idx = np.argmin(np.abs(S_grid - S0))

# Remplir les prix market disponibles sur la grille K
for j, K in enumerate(K_grid):
    # Chercher le strike le plus proche dans les données
    closest_K_call = min(call_prices_by_K.keys(), key=lambda x: abs(x - K), default=None)
    closest_K_put = min(put_prices_by_K.keys(), key=lambda x: abs(x - K), default=None)
    
    if closest_K_call and abs(closest_K_call - K) < 1.0:
        call_prices_market[s0_idx, j] = call_prices_by_K[closest_K_call]
    
    if closest_K_put and abs(closest_K_put - K) < 1.0:
        put_prices_market[s0_idx, j] = put_prices_by_K[closest_K_put]

num_call_prices = np.sum(~np.isnan(call_prices_market))
num_put_prices = np.sum(~np.isnan(put_prices_market))

print(f"\n✓ Prix market mappés sur la grille (T ≈ {target_T} an)")
print(f"  Calls: {num_call_prices} points remplis")
print(f"  Puts:  {num_put_prices} points remplis")

# Visualisation
fig_call_market = plot_heatmap_2d(
    call_prices_market, K_grid, S_grid,
    f"Prix Market Call Options - {TICKER} (T ≈ {target_T}y)",
    "Prix"
)
fig_call_market.show()

fig_put_market = plot_heatmap_2d(
    put_prices_market, K_grid, S_grid,
    f"Prix Market Put Options - {TICKER} (T ≈ {target_T}y)",
    "Prix"
)
fig_put_market.show()

ÉTAPE 2: Construction des heatmaps de prix market

✓ Prix market collectés à S0 = $660.45


## 9. ÉTAPE 3: IV Surface des Données Market (3D)

In [9]:
print("=" * 80)
print("ÉTAPE 3: Construction des surfaces IV market (3D)")
print("=" * 80)

# Grouper IV par strike pour la grille
call_iv_by_K = calls_near_1y.groupby('K')['iv_market'].mean().to_dict()
put_iv_by_K = puts_near_1y.groupby('K')['iv_market'].mean().to_dict()

call_iv_market = np.full((len(S_grid), len(K_grid)), np.nan)
put_iv_market = np.full((len(S_grid), len(K_grid)), np.nan)

# Remplir IV market sur la grille
for j, K in enumerate(K_grid):
    closest_K_call = min(call_iv_by_K.keys(), key=lambda x: abs(x - K), default=None)
    closest_K_put = min(put_iv_by_K.keys(), key=lambda x: abs(x - K), default=None)
    
    if closest_K_call and abs(closest_K_call - K) < 1.0:
        call_iv_market[s0_idx, j] = call_iv_by_K[closest_K_call]
    
    if closest_K_put and abs(closest_K_put - K) < 1.0:
        put_iv_market[s0_idx, j] = put_iv_by_K[closest_K_put]

# Utiliser TOUTES les données pour créer un nuage de points 3D (K, T, IV)
# Projeter avec S = S0 pour la visualisation
print(f"\n✓ Construction des surfaces 3D avec toutes les données...")

# Prendre toutes les données (pas seulement T=1y) pour la surface 3D
# On va afficher (K, T, IV) au lieu de (K, S, IV)
fig_call_iv_market_3d = go.Figure(
    data=go.Scatter3d(
        x=calls_df['K'].values,
        y=calls_df['T'].values,
        z=calls_df['iv_market'].values,
        mode='markers',
        marker=dict(
            size=2,
            color=calls_df['iv_market'].values,
            colorscale='Viridis',
            colorbar=dict(title="IV"),
            showscale=True
        ),
    )
)
fig_call_iv_market_3d.update_layout(
    title=f"IV Surface Market Call - {TICKER} (toutes données)",
    scene=dict(
        xaxis_title="Strike K",
        yaxis_title="Maturité T (années)",
        zaxis_title="IV"
    ),
    width=900, height=700
)
fig_call_iv_market_3d.show()

fig_put_iv_market_3d = go.Figure(
    data=go.Scatter3d(
        x=puts_df['K'].values,
        y=puts_df['T'].values,
        z=puts_df['iv_market'].values,
        mode='markers',
        marker=dict(
            size=2,
            color=puts_df['iv_market'].values,
            colorscale='Viridis',
            colorbar=dict(title="IV"),
            showscale=True
        ),
    )
)
fig_put_iv_market_3d.update_layout(
    title=f"IV Surface Market Put - {TICKER} (toutes données)",
    scene=dict(
        xaxis_title="Strike K",
        yaxis_title="Maturité T (années)",
        zaxis_title="IV"
    ),
    width=900, height=700
)
fig_put_iv_market_3d.show()

print(f"\n✓ Surfaces IV market 3D affichées ({len(calls_df)} calls, {len(puts_df)} puts)")

ÉTAPE 3: Construction des surfaces IV market

✓ IV market extraites



✓ Surfaces IV market affichées


## 10. ÉTAPE 4: Calibration Heston (Neural Network)

In [10]:
print("=" * 80)
print("ÉTAPE 4: Calibration Heston via PyTorch")
print("=" * 80)

calib = calibrate_heston(calls_df, RF_RATE, 0.0, MAX_POINTS, MAX_ITERS, LR)

print("\n" + "=" * 80)
print("PARAMÈTRES HESTON CALIBRÉS")
print("=" * 80)
for k, v in calib.items():
    print(f"  {k:6s} = {v:.6f}")
print("=" * 80)

params_tensor = params_from_calib(calib)

ÉTAPE 4: Calibration Heston via PyTorch
Calibration sur 145 points...
  Iter 20/100 | Loss = 1.351220e+04
  Iter 40/100 | Loss = 1.179529e+04
  Iter 60/100 | Loss = 1.020890e+04
  Iter 80/100 | Loss = 8.754726e+03
  Iter 100/100 | Loss = 7.434344e+03

PARAMÈTRES HESTON CALIBRÉS
  kappa  = 0.455840
  theta  = 0.497250
  sigma  = 1.004332
  rho    = -0.239699
  v0     = 0.485231


/tmp/ipykernel_24367/2774704477.py:44: UserWarning:

Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)



## 11. ÉTAPE 5: Heatmap des Prix Heston (Monte Carlo)

In [11]:
print("=" * 80)
print("ÉTAPE 5: Calcul des prix Heston via Monte Carlo")
print("=" * 80)

call_prices_heston = np.zeros((len(S_grid), len(K_grid)))
put_prices_heston = np.zeros((len(S_grid), len(K_grid)))

total = len(S_grid) * len(K_grid)
count = 0

print(f"\nCalcul de {total} prix (calls + puts)...")

for i, S in enumerate(S_grid):
    for j, K in enumerate(K_grid):
        call_prices_heston[i, j] = heston_monte_carlo_price(
            S, K, T_actual, RF_RATE, 0.0, params_tensor, N_PATHS, N_STEPS, "call"
        )
        put_prices_heston[i, j] = heston_monte_carlo_price(
            S, K, T_actual, RF_RATE, 0.0, params_tensor, N_PATHS, N_STEPS, "put"
        )
        count += 1
        if count % 50 == 0:
            print(f"  Progress: {count}/{total} ({100*count/total:.1f}%)")

print(f"\n✓ Calcul terminé")

# Visualisation
fig_call_heston = plot_heatmap_2d(
    call_prices_heston, K_grid, S_grid,
    f"Prix Heston Call (Monte Carlo) - {TICKER}",
    "Prix"
)
fig_call_heston.show()

fig_put_heston = plot_heatmap_2d(
    put_prices_heston, K_grid, S_grid,
    f"Prix Heston Put (Monte Carlo) - {TICKER}",
    "Prix"
)
fig_put_heston.show()

ÉTAPE 5: Calcul des prix Heston via Monte Carlo

Calcul de 441 prix (calls + puts)...
  Progress: 50/441 (11.3%)
  Progress: 100/441 (22.7%)
  Progress: 150/441 (34.0%)
  Progress: 200/441 (45.4%)
  Progress: 250/441 (56.7%)
  Progress: 300/441 (68.0%)
  Progress: 350/441 (79.4%)
  Progress: 400/441 (90.7%)

✓ Calcul terminé


## 12. ÉTAPE 6: IV Surface BS Inversée (3D)

In [12]:
print("=" * 80)
print("ÉTAPE 6: Calcul des IV BS à partir des prix Heston")
print("=" * 80)

call_iv_heston = np.zeros((len(S_grid), len(K_grid)))
put_iv_heston = np.zeros((len(S_grid), len(K_grid)))

print(f"\nInversion de {total} prix en IV...")
count = 0

for i, S in enumerate(S_grid):
    for j, K in enumerate(K_grid):
        call_iv_heston[i, j] = implied_vol_from_price(
            call_prices_heston[i, j], S, K, T_actual, RF_RATE, "call"
        )
        put_iv_heston[i, j] = implied_vol_from_price(
            put_prices_heston[i, j], S, K, T_actual, RF_RATE, "put"
        )
        count += 1
        if count % 50 == 0:
            print(f"  Progress: {count}/{total} ({100*count/total:.1f}%)")

print(f"\n✓ Inversion terminée")

# Visualisation 3D
fig_call_iv_heston_3d = plot_surface_3d(
    S_grid, K_grid, call_iv_heston,
    f"IV Surface BS Inversée - Call Heston - {TICKER}",
    "IV"
)
fig_call_iv_heston_3d.show()

fig_put_iv_heston_3d = plot_surface_3d(
    S_grid, K_grid, put_iv_heston,
    f"IV Surface BS Inversée - Put Heston - {TICKER}",
    "IV"
)
fig_put_iv_heston_3d.show()

ÉTAPE 6: Calcul des IV BS à partir des prix Heston

Inversion de 441 prix en IV...
  Progress: 50/441 (11.3%)
  Progress: 100/441 (22.7%)
  Progress: 150/441 (34.0%)
  Progress: 200/441 (45.4%)
  Progress: 250/441 (56.7%)
  Progress: 300/441 (68.0%)
  Progress: 350/441 (79.4%)
  Progress: 400/441 (90.7%)

✓ Inversion terminée


## 13. Résumé et Statistiques

In [13]:
print("=" * 80)
print("RÉSUMÉ FINAL")
print("=" * 80)

print(f"\nPrix Heston Call:")
print(f"  Min:  {np.nanmin(call_prices_heston):.4f}")
print(f"  Max:  {np.nanmax(call_prices_heston):.4f}")
print(f"  Mean: {np.nanmean(call_prices_heston):.4f}")

print(f"\nPrix Heston Put:")
print(f"  Min:  {np.nanmin(put_prices_heston):.4f}")
print(f"  Max:  {np.nanmax(put_prices_heston):.4f}")
print(f"  Mean: {np.nanmean(put_prices_heston):.4f}")

print(f"\nIV Call (prix Heston → BS):")
print(f"  Min:  {np.nanmin(call_iv_heston):.4f}")
print(f"  Max:  {np.nanmax(call_iv_heston):.4f}")
print(f"  Mean: {np.nanmean(call_iv_heston):.4f}")

print(f"\nIV Put (prix Heston → BS):")
print(f"  Min:  {np.nanmin(put_iv_heston):.4f}")
print(f"  Max:  {np.nanmax(put_iv_heston):.4f}")
print(f"  Mean: {np.nanmean(put_iv_heston):.4f}")

print("\n" + "=" * 80)
print("✓ ANALYSE COMPLÈTE TERMINÉE")
print("=" * 80)

RÉSUMÉ FINAL

Prix Heston Call:
  Min:  165.6390
  Max:  189.8944
  Mean: 176.9454

Prix Heston Put:
  Min:  152.9065
  Max:  171.6078
  Mean: 162.7529

IV Call (prix Heston → BS):
  Min:  0.6205
  Max:  0.6646
  Mean: 0.6382

IV Put (prix Heston → BS):
  Min:  0.6284
  Max:  0.6492
  Mean: 0.6378

✓ ANALYSE COMPLÈTE TERMINÉE


## 14. Export des Résultats

In [14]:
# Créer DataFrame avec tous les résultats
results = []
for i, S in enumerate(S_grid):
    for j, K in enumerate(K_grid):
        results.append({
            'S': S,
            'K': K,
            'call_price_heston': call_prices_heston[i, j],
            'put_price_heston': put_prices_heston[i, j],
            'call_iv_heston': call_iv_heston[i, j],
            'put_iv_heston': put_iv_heston[i, j],
        })

df_results = pd.DataFrame(results)
filename = f'heston_complete_analysis_{TICKER}.csv'
df_results.to_csv(filename, index=False)

print(f"✓ Résultats exportés vers: {filename}")
display(df_results.head(10))

✓ Résultats exportés vers: heston_complete_analysis_SPY.csv


,S,K,call_price_heston,put_price_heston,call_iv_heston,put_iv_heston
0,650.450012,650.450012,175.160278,159.346013,0.641812,0.634235
1,650.450012,651.450012,173.497374,162.306191,0.636705,0.643564
2,650.450012,652.450012,173.469878,162.050332,0.638072,0.640150
3,650.450012,653.450012,174.504494,161.780378,0.643634,0.636684
4,650.450012,654.450012,170.468880,160.567703,0.629164,0.629504
5,650.450012,655.450012,174.762175,162.963208,0.647574,0.636557
6,650.450012,656.450012,171.309739,163.816456,0.635405,0.637521
7,650.450012,657.450012,173.159324,165.009918,0.644148,0.639820
8,650.450012,658.450012,169.041855,164.303076,0.629387,0.634633
9,650.450012,659.450012,168.091521,164.058742,0.627104,0.631272
